# Learn Logistic Regression by Implementing in NumPy
> Explaination of linear regression by implementing it in Python NumPy.

- toc: false 
- badges: true
- comments: true
- author: Adam Orucu
- categories: [machine learning, numpy]
- image: images/cover/linear-regression.png

In [1]:
#hide
!pip3 install numpy pandas matplotlib seaborn sklearn

In [3]:
#collapse-hide
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Code
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
from sklearn import datasets
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

In the second post in this series we will learn how a logistic regression classfier works and implement it in NumPy. This time we will use the Penguin type dataset

In [20]:
p2data = "https://raw.githubusercontent.com/mcnakhaee/palmerpenguins/master/palmerpenguins/data/penguins.csv"
penguins = pd.read_csv(p2data).dropna()
peng_df = penguins[penguins['sex']=='female'][penguins['species'] != 'Chinstrap'][['species', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'flipper_length_mm']]
peng_df

<ipython-input-20-daf56e11f5e1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  peng_df = penguins[penguins['sex']=='female'][penguins['species'] != 'Chinstrap'][['species', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'flipper_length_mm']]


,species,bill_length_mm,bill_depth_mm,flipper_length_mm,flipper_length_mm
1,Adelie,39.5,17.4,186.0,186.0
2,Adelie,40.3,18.0,195.0,195.0
4,Adelie,36.7,19.3,193.0,193.0
6,Adelie,38.9,17.8,181.0,181.0
12,Adelie,41.1,17.6,182.0,182.0
...,...,...,...,...,...
264,Gentoo,43.5,15.2,213.0,213.0
266,Gentoo,46.2,14.1,217.0,217.0
270,Gentoo,47.2,13.7,214.0,214.0
272,Gentoo,46.8,14.3,215.0,215.0


In [23]:
peng_df['species'] = peng_df['species'].map({'Adelie': 1, 'Gentoo': 2})
peng_df

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,flipper_length_mm
1,1,39.5,17.4,186.0,186.0
2,1,40.3,18.0,195.0,195.0
4,1,36.7,19.3,193.0,193.0
6,1,38.9,17.8,181.0,181.0
12,1,41.1,17.6,182.0,182.0
...,...,...,...,...,...
264,2,43.5,15.2,213.0,213.0
266,2,46.2,14.1,217.0,217.0
270,2,47.2,13.7,214.0,214.0
272,2,46.8,14.3,215.0,215.0


In [ ]:
class LogisticRegressionClassifier:
    """Logistic regression classifier"""
    def __init__(self, r=0.5):
        self.r = r

    def fit(self, X, y, iters=500, lr=0.01):
        """Fits the training data"""
        self.y = y
        self.X = np.column_stack((np.ones((X.shape[0], 1), dtype=X.dtype), X))
        self.m, self.n = X.shape
        self.Theta = np.random.randn(self.n+1, 1)
        loss_prime = lambda x, y, theta: 1/self.m * (self._sigmoid(x @ theta) - y).T @ x
        self._gradient_descent(iters=iters, loss_prime=loss_prime, lr=lr)

    def predict(self, X):
        """Makes prediction"""
        X = np.column_stack((np.ones((X.shape[0], 1), dtype=X.dtype), X))
        return np.array([[1 if res > self.r else -1] for res in self._sigmoid(X @ self.Theta)])

    def _sigmoid(self, Z):
        """Sigmoid function"""
        return np.exp(Z) / (1 + np.exp(Z))
    
    def _gradient_descent(self, iters, loss_prime, lr):
        """Gradient descent algorithm"""
        for i in range(iters):
            grad = loss_prime(self.X, self.y, self.Theta)
            self.Theta -= lr * grad.T